In [1]:
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

import seaborn as sns

Ler o dataset

In [4]:
train_set = pd.read_csv("data/treino_total.csv", sep=',', header=0, low_memory=False)
val_set = pd.read_csv("data/val_total.csv", sep=',', header=0, low_memory=False)
train_set = pd.concat([train_set, val_set])
test_set = pd.read_csv("data/teste_total.csv", sep=',', header=0, low_memory=False)

Pre-processamento básico

In [5]:
# merge das duas ultimas colunas na coluna 'y'
train_set = train_set.drop('IND_BOM_1_2', 1)
train_set = train_set.rename(columns={'IND_BOM_1_1': 'y'})
test_set = test_set.drop('IND_BOM_1_2', 1)
test_set = test_set.rename(columns={'IND_BOM_1_1': 'y'})

# encontra as variavies categoricas
for col in train_set.columns:
    if ((train_set[col].quantile() == 1.0) | (train_set[col].quantile() == 0.0)):
        train_set[col] = train_set[col].astype('category')
for col in test_set.columns:
    if ((test_set[col].quantile() == 1.0) | (test_set[col].quantile() == 0.0)):
        test_set[col] = test_set[col].astype('category')

# Embaralha os datasets
train_set = train_set.sample(frac=1)

In [6]:
X_train = train_set.loc[:, train_set.columns != 'y']
y_train = train_set.loc[:, train_set.columns == 'y']
X_test = test_set.loc[:, test_set.columns != 'y']
y_test = test_set.loc[:, test_set.columns == 'y']

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [12]:
rfr = RandomForestRegressor(random_state = 402)

In [8]:
rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=401)
rf.fit(X_train.values, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=401, verbose=0, warm_start=False)

In [13]:
from sklearn.model_selection import RandomizedSearchCV

Todas as combinações de parametros interessates

In [15]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'bootstrap': [True, False], 'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000], 'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5, 10], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]}


Validação cruzada para encontrar a melhor combinação de hyperparametros
Usando 3-fold

In [19]:
rfr = RandomForestRegressor(random_state = 402)

# procura atraves de 100 combinações diferentes
rfr_random = RandomizedSearchCV(estimator=rfr, param_distributions=random_grid,\
                                n_iter = 100, scoring='neg_mean_absolute_error',\
                                cv = 3, verbose=2, random_state=402, n_jobs=-1,\
                                return_train_score=True)

# Fit a busca aleatoria
rfr_random.fit(X_train.values, y_train.values.ravel());

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] bootstrap=True, n_estimators=1366, min_samples_leaf=4, max_features=auto, min_samples_split=2, max_depth=40 
[CV] bootstrap=True, n_estimators=1366, min_samples_leaf=4, max_features=auto, min_samples_split=2, max_depth=40 
[CV] bootstrap=True, n_estimators=1366, min_samples_leaf=4, max_features=auto, min_samples_split=2, max_depth=40 
[CV] bootstrap=True, n_estimators=1155, min_samples_leaf=2, max_features=sqrt, min_samples_split=5, max_depth=70 
[CV]  bootstrap=True, n_estimators=1155, min_samples_leaf=2, max_features=sqrt, min_samples_split=5, max_depth=70, total= 5.6min
[CV] bootstrap=True, n_estimators=1155, min_samples_leaf=2, max_features=sqrt, min_samples_split=5, max_depth=70 
[CV]  bootstrap=True, n_estimators=1155, min_samples_leaf=2, max_features=sqrt, min_samples_split=5, max_depth=70, total= 5.5min
[CV] bootstrap=True, n_estimators=1155, min_samples_leaf=2, max_features=sqrt, min_samples_split=5, max_depth

KeyboardInterrupt: 